<a href="https://colab.research.google.com/github/BeltranPascual/Deep-Learning/blob/master/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Red neuronal recurrente
# Parte 1 - Preprocesamiento de datos
# Importación de las bibliotecas

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importando el set de entrenamiento

In [0]:
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

# Escalado de características

In [0]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creación de una estructura de datos con 60 pasos de tiempo y 1 salida

In [0]:
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)


# Remodelación

In [0]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Parte 2 - Construyendo el RNN

# Importación de las librerías y paquetes Keras

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Inicializando el RNN

In [0]:
regressor = Sequential()

# Añadiendo la primera capa de LSTM y algo de regularización de la deserción

In [0]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Añadiendo la segunda capa de LSTM y algo de regularización de la deserción

In [0]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Añadiendo la tercera capa de LSTM y algo de regularización de la deserción

In [0]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))


# Añadiendo la cuarta capa de LSTM y algo de regularización de la deserción

In [0]:
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Añadiendo la capa de salida

In [0]:
regressor.add(Dense(units = 1))

# Compilación del RNN

In [0]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Ajustar el RNN al set de entrenamiento

In [0]:
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

# Parte 3 - Hacer las predicciones y visualizar los resultados

# Obtención del precio real de las acciones de 2017

In [0]:
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

# Obtención del precio de las acciones previsto para 2017

In [0]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Visualización de los resultados

In [0]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()